In [1]:
import tree  # pip install dm_tree
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.env.single_agent_env_runner import SingleAgentEnvRunner

# Configure the EnvRunners.
config = (
    PPOConfig()
    .environment("Acrobot-v1")
    .env_runners(num_env_runners=2, num_envs_per_env_runner=1)
)
# Create the EnvRunner actors.
env_runners = [
    ray.remote(SingleAgentEnvRunner).remote(config=config)
    for _ in range(config.num_env_runners)
]

# Gather lists of `SingleAgentEpisode`s (each EnvRunner actor returns one
# such list with exactly two episodes in it).
episodes = ray.get([
    er.sample.remote(num_episodes=3)
    for er in env_runners
])
# Two remote EnvRunners used.
assert len(episodes) == 2
# Each EnvRunner returns three episodes
assert all(len(eps_list) == 3 for eps_list in episodes)

# Report the returns of all episodes collected
for episode in tree.flatten(episodes):
    print("R=", episode.get_return())

2025-07-15 13:09:34,439	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 
(SingleAgentEnvRunner pid=96298) 2025-07-15 13:09:39,622	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!


R= -500.0
R= -500.0
R= -500.0
R= -500.0
R= -388.0
R= -500.0


In [2]:
from ray.rllib.algorithms.ppo import PPOConfig

# Configure and build an initial algorithm.
config = (
    PPOConfig()
    .environment("Pendulum-v1")
)
ppo = config.build()

# Train for one iteration, then save to a checkpoint.
print(ppo.train())
checkpoint_dir = ppo.save_to_path()
print(f"saved algo to {checkpoint_dir}")

2025-07-15 14:06:33,004	WARNING deprecation.py:50 -- DeprecationWarning: `build` has been deprecated. Use `AlgorithmConfig.build_algo` instead. This will raise an error in the future!
2025-07-15 14:06:33,010	WARNING algorithm_config.py:4703 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
/opt/anaconda3/envs/rl/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:512: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/opt/anaconda3/envs/rl/lib

{'timers': {'training_iteration': 2.279638792038895, 'restore_env_runners': 1.2083910405635834e-05, 'training_step': 2.2795353330438957, 'env_runner_sampling_timer': 0.4964607080910355, 'learner_update_timer': 1.7798381249886006, 'synch_weights': 0.0030837920494377613}, 'env_runners': {'episode_return_mean': -1560.5890724657588, 'weights_seq_no': 0.0, 'timers': {'connectors': {'TensorToNumpy': 1.2900714393356271e-05, 'NormalizeAndClipActions': 1.7433756679771867e-05, 'AddTimeDimToBatchAndZeroPad': 2.09595676856077e-06, 'BatchIndividualItems': 5.527426389743975e-06, 'NumpyToTensor': 8.549708406301404e-06, 'AddObservationsFromEpisodesToBatch': 2.74782993432141e-06, 'UnBatchToIndividualItems': 5.518644705516158e-06, 'AddStatesFromEpisodesToBatch': 1.541302740607286e-06, 'ListifyDataForVectorEnv': 9.113648814768929e-06, 'GetActions': 3.490234622012163e-05, 'RemoveSingleTsTimeRankFromBatch': 4.996858600128309e-07}}, 'episode_return_max': -1351.899365154296, 'num_module_steps_sampled_lifetim